In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import matplotlib.pyplot as plt
from itertools import islice

from torchlensmaker.raytracing import super_refraction, clamped_refraction, ray_point_squared_distance, refraction
from torchlensmaker.optics import FocalPointLoss, ParallelBeamUniform, ParallelBeamRandom, FixedGap, RefractiveSurface, OpticalStack, Anchor
from torchlensmaker.shapes import Parabola, PiecewiseLine, Line, CircularArc, BezierSpline
from torchlensmaker.training import render, optimize

In [ ]:

# Demo using surface anchors for surfaces

lens_radius = 60. # lens radius in mm
surface1 = CircularArc(lens_radius, [-200.])
surface2 = surface1.share(scale=-1.0)

optics = OpticalStack([
    ParallelBeamUniform(radius=0.9*lens_radius),
    FixedGap(20.),
    
    RefractiveSurface(surface1, (1.0, 1.49), anchors=(Anchor.Center, Anchor.Edge)),
    FixedGap(0.1),
    RefractiveSurface(surface2, (1.49, 1.0), anchors=(Anchor.Edge, Anchor.Center)),

    FixedGap(0.1),
    
    RefractiveSurface(surface1, (1.0, 1.49), anchors=(Anchor.Center, Anchor.Edge)),
    FixedGap(0.1),
    RefractiveSurface(surface2, (1.49, 1.0), anchors=(Anchor.Edge, Anchor.Center)),
    
    FixedGap(200.0), # focal length
    FocalPointLoss(),
])


render(optics, num_rays=10)

optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    num_rays = 20,
    num_iter = 150
)

render(optics, num_rays=10)

